<a href="https://colab.research.google.com/github/junieberry/DL-fromScratch2/blob/main/ToyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Project : 한국어 영화 리뷰 감정 분석

### issue
- 정규 표현식

In [18]:
!pip install konlpy
%env JAVA_HOME "C:\Program Files\Java\jdk-14.0.2"

env: JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


In [47]:
import numpy as np
import pandas as pd
import konlpy
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


## 데이터 column 정보

- id : naver에서 제공하는 id
- document : 리뷰
- label : 레이블(0 : negative, 1: positive)

In [20]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [21]:
## 필드 구문이 탭으로 되어 있어서 \t로 구분자를 지정해준다.
train_data = pd.read_csv("nsmc/ratings_train.txt","\t")
test_data = pd.read_csv("nsmc/ratings_test.txt","\t")

print(train_data.shape)
print(test_data.shape)

(150000, 3)
(50000, 3)


In [22]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 데이터 정제
### 중복 제거


In [23]:
## document 열에서 중복 제거한 샘플의 개수, label 열에서 중복 제거한 샘플의 개수
train_data['id'].nunique(), train_data['document'].nunique(), train_data['label'].nunique()

(150000, 146182, 2)

In [24]:
## 중복 데이터 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
test_data.drop_duplicates(subset=['document'], inplace=True)

print('중복 제거된 train data >> ',len(train_data))
print('중복 제거된 test data >> ',len(test_data))

중복 제거된 train data >>  146183
중복 제거된 test data >>  49158


### Null 제거

In [25]:
## 널값 확인
print(train_data.isnull().sum())
print(test_data.isnull().sum())

id          0
document    1
label       0
dtype: int64
id          0
document    1
label       0
dtype: int64


In [26]:
## Null 값이 존재하는 행 제거
train_data = train_data.dropna(how = 'any')
test_data = test_data.dropna(how = 'any')

print(len(train_data))
print(len(test_data))

146182
49157


## 데이터 전처리

In [27]:
## 정규 표현식을 사용해 한글만 남기고 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

## white space data나 empty data를 Null으로 변경하고 제거
train_data['document'] = train_data['document'].str.replace("^ +","")
train_data['document'].replace('',np.nan,inplace=True)
train_data = train_data.dropna(how = 'any')

test_data['document'] = test_data['document'].str.replace("^ +","")
test_data['document'].replace('',np.nan,inplace=True)
test_data = test_data.dropna(how = 'any')

print(len(train_data))
print(len(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


145393
48852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


### Tokenize

불용어 제거

- 불용어는 정의하기 나름이다!

In [28]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [29]:
## KoNLPy에서 제공하는 형태소 분석기
## stem 옵션으로 정규화 수행
okt = Okt()

print("okt.morph >> ",okt.morphs(u"우리는 파이썬에서 정한 규칙을 벗어났기 때문에 에러를 만나게 됩니다.", stem = True))
print("okt.nouns >> ", okt.nouns(u"파이썬은 에러가 발생한경우에 굉장히 친절하게 알려주는 언어입니다., stem = True"))
print("okt.phrases >>" , okt.phrases(u'파이썬 에러 발생한 문장을 보면, 에러가 발생한 파일이름과 몇번째 줄인지, 그리고 에러가 발생한 문장도 함께 보여줍니다.'))
print("okt.pos >>", okt.pos(u"에러를 만난다면, 당황하지 않고 먼저 읽으시기 바랍니다.", stem = True))
print("okt.pos >>", okt.pos(u"ㅋㅋㅋㅋ야 이거 실화냐? 개꿀잼", stem = True))
print("okt.pos >>", okt.pos(u"그렇게 하면 안돼", stem = True))
print("okt.pos >>", okt.pos(u"그러케 하면 않되", stem = True))

okt.morph >>  ['우리', '는', '파이썬', '에서', '정', '한', '규칙', '을', '벗어나다', '때문', '에', '에러', '를', '만나다', '되다', '.']
okt.nouns >>  ['파이썬', '에러', '발생', '경우', '언어']
okt.phrases >> ['파이썬', '파이썬 에러', '파이썬 에러 발생', '문장', '에러', '발생', '파일이름', '파일이름과 몇번째', '파일이름과 몇번째 줄', '몇번째']
okt.pos >> [('에러', 'Noun'), ('를', 'Josa'), ('만나다', 'Verb'), (',', 'Punctuation'), ('당황', 'Noun'), ('하다', 'Verb'), ('않다', 'Verb'), ('먼저', 'Noun'), ('읽다', 'Verb'), ('바라다', 'Verb'), ('.', 'Punctuation')]
okt.pos >> [('ㅋㅋㅋㅋ', 'KoreanParticle'), ('야', 'Exclamation'), ('이', 'Determiner'), ('거', 'Noun'), ('실화', 'Noun'), ('냐', 'Josa'), ('?', 'Punctuation'), ('개꿀잼', 'Noun')]
okt.pos >> [('그렇게', 'Adverb'), ('하다', 'Verb'), ('안', 'VerbPrefix'), ('돼다', 'Verb')]
okt.pos >> [('그러케', 'Noun'), ('하다', 'Verb'), ('않다', 'Verb')]


In [17]:
## 불용어 제거
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

X_test = []
for sentence in test_data['document']:
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

print(X_train[0])
print(X_test[0])

['아', '더빙', '진짜', '짜증나다', '목소리']
['굳다', 'ㅋ']


### 정수 인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

## 전체 훈련 데이터에서 등장 빈도가 높은 순서대로 부여됨
print(tokenizer.word_index)

In [31]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 개수 :',total_cnt)
print('%s번 이하로 등장하는 단어의 수: %s'%(threshold - 1, rare_cnt))
print("희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 개수 : 43752
2번 이하로 등장하는 단어의 수: 24337
희귀 단어의 비율: 55.62488571950996
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8715872104872904


In [32]:
## 2번 이하로 등장하는 단어가 전체 단어의 55%나 차지하지만 등장 비율은 2%도 안됨 ==> 제거하자
vocab_size = total_cnt - rare_cnt +1
vocab_size

19416

In [33]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

### 빈 샘플 제거

In [34]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

145162
145162


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


### 패딩

샘플들의 길이 동일하게 맞춰주기

In [ ]:
print('리뷰 최대 길이 >> ', max(len(l) for l in X_train))
print('리뷰 평균 길이 >> ', sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [35]:
max_len=30
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# LSTM으로 분류하기

In [36]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [37]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [38]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [39]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1936/1936 [==============================] - 177s 88ms/step - loss: 0.3895 - acc: 0.8224 - val_loss: 0.3699 - val_acc: 0.8416

Epoch 00001: val_acc improved from -inf to 0.84159, saving model to best_model.h5
Epoch 2/15
1936/1936 [==============================] - 170s 88ms/step - loss: 0.3283 - acc: 0.8570 - val_loss: 0.3330 - val_acc: 0.8546

Epoch 00002: val_acc improved from 0.84159 to 0.85461, saving model to best_model.h5
Epoch 3/15
1936/1936 [==============================] - 171s 89ms/step - loss: 0.3018 - acc: 0.8713 - val_loss: 0.3302 - val_acc: 0.8580

Epoch 00003: val_acc improved from 0.85461 to 0.85799, saving model to best_model.h5
Epoch 4/15
1936/1936 [==============================] - 169s 87ms/step - loss: 0.2833 - acc: 0.8819 - val_loss: 0.3305 - val_acc: 0.8571

Epoch 00004: val_acc did not improve from 0.85799
Epoch 5/15
1936/1936 [==============================] - 171s 88ms/step - loss: 0.2672 - acc: 0.8903 - val_loss: 0.3295 - val_acc: 0.8576

Epoch 00

### 테스트해보기

In [40]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1527/1527 [==============================] - 22s 14ms/step - loss: 0.3419 - acc: 0.8566

 테스트 정확도: 0.8566


### 리뷰 예측해보기

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [42]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')
sentiment_predict('흠... 잘 모르겠다')
sentiment_predict('영화 안 본 눈 삽니다..')
sentiment_predict('돈 아깝다')

95.78% 확률로 긍정 리뷰입니다.

89.90% 확률로 부정 리뷰입니다.

72.25% 확률로 부정 리뷰입니다.

98.96% 확률로 부정 리뷰입니다.



In [43]:
sentiment_predict('아 열받아')

74.11% 확률로 부정 리뷰입니다.

74.11% 확률로 부정 리뷰입니다.



In [44]:
sentiment_predict('이 라면 개꿀딱')

53.26% 확률로 긍정 리뷰입니다.



In [45]:
sentiment_predict('황정민 못생겼다')

83.88% 확률로 부정 리뷰입니다.



In [46]:
sentiment_predict('화질 쌉그지네')

74.60% 확률로 부정 리뷰입니다.

